In [26]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, linregress

In [2]:
fcst = pd.read_excel('../Excel/20210210_MVCM_Car sales fcst.xlsm')
fcst

,Year,Month,t,Units,T*C
0,2009.0,1,1,69664,NaN
1,NaN,2,2,61579,NaN
2,NaN,3,3,64242,NaN
3,NaN,4,4,51395,NaN
4,NaN,5,5,53440,NaN
...,...,...,...,...,...
127,NaN,8,128,108074,NaN
128,NaN,9,129,100757,NaN
129,NaN,10,130,107110,NaN
130,NaN,11,131,124804,NaN


In [7]:
fcst_c = fcst[['Month','t','Units','T*C']]
fcst_c = fcst_c.dropna().reset_index(drop=True)
fcst_c

,Month,t,Units,T*C
0,7,7,56443,62677.083333
1,8,8,58926,62357.875000
2,9,9,58505,62320.833333
3,10,10,67882,62746.208333
4,11,11,64914,63464.083333
...,...,...,...,...
115,2,122,104009,114119.958333
116,3,123,117529,113055.625000
117,4,124,98366,112029.666667
118,5,125,102422,111203.375000


In [13]:
fcst_c['S*I'] = fcst_c['Units']/fcst_c['T*C']
fcst_c

,Month,t,Units,T*C,S*I
0,7,7,56443,62677.083333,0.900536
1,8,8,58926,62357.875000,0.944965
2,9,9,58505,62320.833333,0.938771
3,10,10,67882,62746.208333,1.081850
4,11,11,64914,63464.083333,1.022846
...,...,...,...,...,...
115,2,122,104009,114119.958333,0.911401
116,3,123,117529,113055.625000,1.039568
117,4,124,98366,112029.666667,0.878035
118,5,125,102422,111203.375000,0.921033


In [18]:
n = 12
ies = fcst_c[['Month','S*I']]
ies = ies.rename(columns={'S*I':'S'})
ies = ies.groupby('Month', as_index=False).mean()
suma = ies['S'].sum()
ies['S Aj'] = ies['S']/suma*n
ies = ies[['Month','S Aj']]
ies

,Month,S Aj
0,1,0.957382
1,2,0.908366
2,3,0.989960
3,4,0.889992
4,5,0.945041
5,6,0.952528
6,7,0.951830
7,8,0.995334
8,9,0.942766
9,10,1.012921


In [25]:
fcst_m = pd.merge(fcst_c,ies).sort_values(by='t').reset_index(drop=True)
fcst_m

,Month,t,Units,T*C,S*I,S Aj
0,7,7,56443,62677.083333,0.900536,0.951830
1,8,8,58926,62357.875000,0.944965,0.995334
2,9,9,58505,62320.833333,0.938771,0.942766
3,10,10,67882,62746.208333,1.081850,1.012921
4,11,11,64914,63464.083333,1.022846,1.096314
...,...,...,...,...,...,...
115,2,122,104009,114119.958333,0.911401,0.908366
116,3,123,117529,113055.625000,1.039568,0.989960
117,4,124,98366,112029.666667,0.878035,0.889992
118,5,125,102422,111203.375000,0.921033,0.945041
